In [1]:
from __future__ import print_function
import ipywidgets as widgets
import plotly.graph_objects as go
from epynet import Network

In [2]:
!ls ../water_networks/

anytown_master.inp  d-town_master.inp  from_Exeter_db
anytown_master.rpt  d-town_master.rpt


In [3]:
wds = Network('../water_networks/anytown_master.inp')
wds.solve()

# Topology of Anytown

In [4]:
def assemble_2d_traces(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)

    junc_trace = go.Scatter(
        x = junc_x,
        y = junc_y,
        mode='markers'
        )

    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(None)

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(None)

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(None)

    pipe_trace = go.Scatter(
        x = pipe_x,
        y = pipe_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [5]:
def assemble_3d_traces(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)

    junc_trace = go.Scatter3d(
        x = junc_x,
        y = junc_y,
        z = junc_z,
        mode='markers'
        )

    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(None)

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(None)

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(None)

    pipe_trace = go.Scatter3d(
        x = pipe_x,
        y = pipe_y,
        z = pipe_z,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [6]:
junc_trace, pipe_trace = assemble_2d_traces(wds)

junc_trace.marker = dict(
    showscale = True,
    colorscale = 'YlGnBu',
    reversescale = True,
    color = list(wds.junctions.head),
    size = 10,
    colorbar = dict(
        thickness = 15,
        title = 'Head, m',
        xanchor = 'left',
        titleside = 'right'
    ))

fig = go.FigureWidget(
    data = [pipe_trace],
    layout = go.Layout(
        title = 'Anytown',
        titlefont_size = 16,
        showlegend = False,
        hovermode = 'closest',
        margin = dict(b=20,l=5,r=5,t=40),
        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False))
        )

In [7]:
w = widgets.IntSlider(value=75)
w

IntSlider(value=75)

In [8]:
button = widgets.Button(description="Click Me!")

display(button)

def on_button_clicked(b):
    global fig
    wds.pumps.speed = wds.pumps.speed * w.value / 100 + .35
    wds.solve()
    junc_trace.marker.color = list(wds.junctions.head)
    #fig.add_trace(pipe_trace)
    fig.add_trace(junc_trace)
    fig.plotly_update()

button.on_click(on_button_clicked)
display(widgets.VBox([fig]))

Button(description='Click Me!', style=ButtonStyle())

    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'widt…

# Bokeh version

In [20]:
from bokeh import plotting as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [21]:
p = plt.figure()

In [22]:
p.line(junc_trace['x'], junc_trace['y'])

GlyphRenderer(id='1330', ...)

In [23]:
plt.show(p)

# Bokeh interactive example

In [24]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [25]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

In [26]:
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [27]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [28]:
show(p, notebook_handle=True)

In [29]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

interactive(children=(Dropdown(description='f', options=('sin', 'cos'), value='sin'), IntSlider(value=1, descr…

<function __main__.update(f, w=1, A=1, phi=0)>